In [19]:
with open("input.txt", "r") as file:
    content = file.readlines()


In [20]:
def str_clean(sent):

    append_to_prev = False

    if len(sent) > 0 and not sent[0] =='[':
        append_to_prev = True
        sent = sent[:-1]
        return append_to_prev, sent

    else:
        sent = sent.split(":")[2:]

        if len(sent) == 0 or sent[-1].isspace():
            return None, None
        
        sent[-1] = sent[-1][:-1]
        sent = ":".join(sent)
        return append_to_prev, sent


content_clean = []
clean_index = []
for i, c in enumerate(content):
    append_to_prev, sent = str_clean(c)
    if sent is not None:
        if append_to_prev:
            content_clean[-1] += ' \n' + sent
        else:
            content_clean.append(sent)
            clean_index.append(i)



In [22]:
import re
import requests

import torch
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification



### For ner

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
ner_nlp = pipeline("ner", model=model, tokenizer=tokenizer)



### For query_llm 
# (login to HF using 'huggingface-cli login --token $HUGGINGFACE_TOKEN' before this)
model_llm = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_llm)
chat_llm = pipeline(
    "text-generation",
    model=model_llm,
    torch_dtype=torch.float16,
    device_map="auto",
)


### For chain_of_density_summarise (we're using the regular summariser for now)
summarizer = pipeline("summarization")




def get_context_from_links(text):
    context = ''
    urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
    for url in urls:
        f = requests.get(url)
        context += '\n {}'.format(f.text)
    return chain_of_density_summarise(context)



def chain_of_density_summarise(prev, next):
    # Regular summariser for now
    summarized = summarizer(prev + next, min_length=0, max_length=300)
    return summarized



def query_llm(prompt):
    sequences = chat_llm(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,
    )
    
    return sequences[0]['generated_text']

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


OSError: You are trying to access a gated repo.
Make sure to request access at https://huggingface.co/meta-llama/Llama-2-7b-chat-hf and pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`.

In [15]:
summaries = []

print('total entries: {}'.format(len(content_clean)))

# for i in range(len(content_clean)):
for i in range(10):

    print('working on entry {}'.format(i))

    ner_content = ner_nlp(content_clean[i])
    overall_context = content_clean[i]

    for d in ner_content:
        prompt = "What is {} ? \nContext: {}. Keep it very brief.".format(d['word'], content_clean[i])
        response = query_llm(prompt)
        overall_context += '\n' + response

    context_from_links = get_context_from_links(content_clean[i])
    overall_context += "\n\n" + context_from_links
        

    added = False
    for s in summaries:
        prompt = "Are these things part of the same conversation ? {} and {}. Strictly reply in 'yes' or 'no'.".format(s['content'], overall_context)
        response = query_llm(prompt)

        if 'yes' in response:
            s['content'] = chain_of_density_summarise(s['content'], overall_context)
            s['index'].append(clean_index[i])
            added = True
        else:
            pass

    if added is False:
        s_new = {
            'content': chain_of_density_summarise('', overall_context),
            'index': [clean_index[i]]
        }
        summaries.append(s_new)


total entries: 118
working on entry 0
working on entry 1
working on entry 2
working on entry 3
working on entry 4
working on entry 5
working on entry 6
working on entry 7
working on entry 8
working on entry 9
working on entry 10
working on entry 11
working on entry 12
working on entry 13
working on entry 14
working on entry 15
working on entry 16
working on entry 17
working on entry 18
working on entry 19
working on entry 20
working on entry 21
working on entry 22
working on entry 23
working on entry 24
working on entry 25
working on entry 26
working on entry 27
working on entry 28
working on entry 29
working on entry 30
working on entry 31
working on entry 32
working on entry 33
working on entry 34
working on entry 35
working on entry 36
working on entry 37
working on entry 38
working on entry 39
working on entry 40
working on entry 41
working on entry 42
working on entry 43
working on entry 44
working on entry 45
working on entry 46
working on entry 47
working on entry 48
working on 